In [13]:
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
#ścieka do folderu
dataset = '/Users/joannawitek/Desktop/DL-project/Fruits'

In [4]:
#ścieki do folderów train i test
train_folder = '/Users/joannawitek/Desktop/DL-project/Fruits-train'
test_folder = '/Users/joannawitek/Desktop/DL-project/Fruits-test'

In [5]:
#stworzenie folderów train i test
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

In [6]:
#funkcja która przesunie pliki do folderów train i test
def move_files(file_list, target_folder):
    for file_path in file_list:
        filename = os.path.basename(file_path)
        dst = os.path.join(target_folder, filename)
        shutil.copy(file_path, dst)

In [7]:
#lista subfolderów (kategorii owoców)
subfolders = [subdir for subdir in os.listdir(dataset) if os.path.isdir(os.path.join(dataset, subdir))]

In [8]:
#tworzenie subfolderów w folderach train i jest dla kadej kategorii owoców
for subdir in subfolders:
    os.makedirs(os.path.join(train_folder, subdir), exist_ok=True)
    os.makedirs(os.path.join(test_folder, subdir), exist_ok=True)

    #lista plików ze zdjęciami dla kadej kategorii owoców
    fruit_folder = os.path.join(dataset, subdir)
    image_files = [os.path.join(fruit_folder, file) for file in os.listdir(fruit_folder)]

    #podział obrazów na zbiory train i test
    train_files, test_files = train_test_split(image_files, test_size=0.3, random_state=42)

    #przesunicie plików do folderu train
    move_files(train_files, os.path.join(train_folder, subdir))

    #przesunięcie plików do folderu test
    move_files(test_files, os.path.join(test_folder, subdir))

In [9]:
#ustawienie rozmiaru obrazu i innych parametrów do augumentacji
image_size = (320, 258)
batch_size = 32
rotation_range = 20
zoom_range = 0.2
horizontal_flip = True

In [10]:
#image data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=rotation_range,
    zoom_range=zoom_range,
    horizontal_flip=horizontal_flip
)

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_folder,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 31076 images belonging to 15 classes.
Found 13330 images belonging to 15 classes.


In [14]:
#stworzenie modelu sekwencyjnego
model = Sequential()

#dodanie warstwy konwolucyjnej - 32 filtry, kernel 3x3, aktywacja 'relu'
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(320, 258, 3)))

#dodanie warstwy max pooling - pool size 2x2 
model.add(MaxPooling2D(pool_size=(2, 2)))

# dodanie kolejnej warstwy konwolucyjnej - 64 filtry, kernel 3x3, aktywacja 'relu'
model.add(Conv2D(64, (3, 3), activation='relu'))

#dodanie kolejnej warstwy max pooling - pool size 2x2 
model.add(MaxPooling2D(pool_size=(2, 2)))

#spłaszczenie outputu z poprzedniej warstwy
model.add(Flatten())

#dodanie warstwy fully connected - 128 jednostek, aktywacja 'relu'
model.add(Dense(128, activation='relu'))

#dodanie dropoutu by zapobiec overfitingowi
model.add(Dropout(0.5))

#dodanie warstwy wyjciowej z liczbą klas i aktywacją 'softmax'
model.add(Dense(15, activation='softmax'))

#skompilowanie modelu z lossem 'categorical_crossentropy' i optymalizatorem 'adam'
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
#trenowanie modelu
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

/var/folders/zl/32rqypm507n6c2_lb0kw70500000gn/T/ipykernel_15993/3121286001.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10


2023-06-08 15:43:55.483968: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


971/971 [==============================] - 775s 798ms/step - loss: 1.1988 - accuracy: 0.6042 - val_loss: 0.4978 - val_accuracy: 0.8487
Epoch 2/10
971/971 [==============================] - 843s 868ms/step - loss: 0.6387 - accuracy: 0.7817 - val_loss: 0.2577 - val_accuracy: 0.9165
Epoch 3/10
971/971 [==============================] - 831s 855ms/step - loss: 0.5148 - accuracy: 0.8269 - val_loss: 0.2527 - val_accuracy: 0.9117
Epoch 4/10
971/971 [==============================] - 863s 888ms/step - loss: 0.4478 - accuracy: 0.8475 - val_loss: 0.1847 - val_accuracy: 0.9363
Epoch 5/10
971/971 [==============================] - 1063s 1s/step - loss: 0.4151 - accuracy: 0.8614 - val_loss: 0.1621 - val_accuracy: 0.9453
Epoch 6/10
971/971 [==============================] - 871s 897ms/step - loss: 0.3557 - accuracy: 0.8819 - val_loss: 0.1580 - val_accuracy: 0.9472
Epoch 7/10
971/971 [==============================] - 902s 929ms/step - loss: 0.3285 - accuracy: 0.8906 - val_loss: 0.1240 - val_accuracy

In [17]:
#ewaluacja modelu na zbiorze testowym
score = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

416/416 [==============================] - 67s 162ms/step - loss: 0.0877 - accuracy: 0.9702
Test loss: 0.08770725131034851
Test accuracy: 0.9701772928237915


In [18]:
#zapisanie modelu
model.save('fruit_recognition_model.h5')